In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/diets.csv
/kaggle/input/medications.csv
/kaggle/input/Symptom-severity.csv
/kaggle/input/workout_df.csv
/kaggle/input/precautions_df.csv
/kaggle/input/Training.csv
/kaggle/input/description.csv
/kaggle/input/symtoms_df.csv


In [4]:
import  pandas as pd
from sklearn.metrics import confusion_matrix


******Checking the report of Data******

Using the ydataprofilling Technique it gives the report of data in HTML format to check whether the data having missing values, Imbalacing of data or some corellation of data...


In [ ]:
dataset = pd.read_csv('/kaggle/input/Training.csv')
!pip install numba==0.56.4
!pip install -U ydata-profiling visions numba
!pip uninstall ydata-profiling visions numba -y
!pip install ydata-profiling
!pip show ydata-profiling visions numba

This code Snippet gives the report

In [ ]:
from ydata_profiling import ProfileReport
profile = ProfileReport(dataset , title="Profiling Report")
profile.to_file("profiling_report.html")

Labeling of Target and Splitting of data

In [8]:

X=dataset.drop(labels=['prognosis'], axis=1)
y=dataset['prognosis']
# separate dataset into train and test
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
le.fit(y)
Y = le.transform(y)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    dataset.drop(labels=['prognosis'], axis=1),
    dataset['prognosis'],
    test_size=0.3,
    random_state=0)

In [18]:
X_train

,itching,skin_rash,nodal_skin_eruptions,continuous_sneezing,shivering,chills,joint_pain,stomach_pain,acidity,ulcers_on_tongue,...,pus_filled_pimples,blackheads,scurring,skin_peeling,silver_like_dusting,small_dents_in_nails,inflammatory_nails,blister,red_sore_around_nose,yellow_crust_ooze
3972,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1990,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
607,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
296,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
587,0,1,0,0,0,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4859,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3264,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1653,0,0,0,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2607,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


**Plotting the data which have highly correlated features**

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

#Using Pearson Correlation
corrmat = X_train.corr()
fig, ax = plt.subplots()
fig.set_size_inches(11,11)
sns.heatmap(corrmat)

**Using Feature Selection Technique**

In [ ]:
from sklearn.feature_selection import mutual_info_classif
# determine the mutual information
mutual_info = mutual_info_classif(X_train, y_train)
mutual_info

In [ ]:
mutual_info = pd.Series(mutual_info)
mutual_info.index = X_train.columns
mutual_info.sort_values(ascending=False)

In [ ]:
mutual_info.sort_values(ascending=False).plot.bar(figsize=(20, 8))

In [ ]:
import pandas as pd
from sklearn.feature_selection import SelectKBest

# Fit SelectKBest
sel_five_cols = SelectKBest(mutual_info_classif, k=30)
sel_five_cols.fit(X_train, y_train)

# Get feature scores
feature_scores = pd.DataFrame({
    'Feature': X_train.columns,
    'Score': sel_five_cols.scores_
})

# Sort by score to see the importance
feature_scores = feature_scores.sort_values(by='Score', ascending=False)

# Display the top 30 features
print(feature_scores.head(30))


In [ ]:
selected_features = X_train.columns[sel_five_cols.get_support()]
print(f"Top {len(selected_features)} Selected Features:")
print(selected_features.tolist())
assert len(selected_features) == 30, "The number of selected features is not 25!"


In [ ]:
# Select the top 30 features from training and test sets
X_train_selected = X_train.loc[:, sel_five_cols.get_support()]
X_test_selected = X_test.loc[:, sel_five_cols.get_support()]
X_train_selected = X_train_selected.astype(float)
X_test_selected = X_test_selected.astype(float)
print(X_train_selected.dtypes)
print(X_test_selected.dtypes)




In [ ]:
import numpy as np

def to_float_numpy(array):
    if isinstance(array, np.ndarray):
        return array.astype(float)
    else:  # pandas DataFrame
        return array.to_numpy(dtype=float)

X_train_selected = to_float_numpy(X_train_selected)
X_test_selected = to_float_numpy(X_test_selected)

# No need to call to_numpy() or values again
print(type(X_train_selected))  # <class 'numpy.ndarray'>
print(X_train_selected.dtype)  # float64 (or float32)


In [ ]:
from ydata_profiling import ProfileReport
profile = ProfileReport(X_train_selected , title="Profiling Report")
profile.to_file("Selected_profiling_report.html")

In [ ]:
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, confusion_matrix
import numpy as np


# Create a dictionary to store models
models = {
    'SVC': SVC(kernel='linear'),
    'RandomForest': RandomForestClassifier(n_estimators=100, random_state=42),
    'GradientBoosting': GradientBoostingClassifier(n_estimators=100, random_state=42),
    'KNeighbors': KNeighborsClassifier(n_neighbors=5),
    'MultinomialNB': MultinomialNB()
}

for model_name, model in models.items():
    # Train the model
    model.fit(X_train_selected, y_train)

    # Test the model
    predictions = model.predict(X_test_selected)

    # Calculate accuracy
    accuracy = accuracy_score(y_test, predictions)
    print(f"{model_name} Accuracy: {accuracy}")

    # Calculate confusion matrix
    cm = confusion_matrix(y_test, predictions)
    print(f"{model_name} Confusion Matrix:")
    print(np.array2string(cm, separator=', '))

    print("\n" + "="*40 + "\n")


In [18]:
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train,y_train)
ypred = rf.predict(X_test)
test_accuracy = accuracy_score(y_test,ypred)

In [ ]:
train_pred = rf.predict(X_train)
train_accuracy = accuracy_score(y_train, train_pred)
train_accuracy
if train_accuracy - test_accuracy > 0.1:  # 10% gap threshold
    print("The model is likely overfitting.")
else:
    print("The model generalizes well.")

In [ ]:
confusion_matrix(y_train,train_pred)

In [ ]:
import pickle

with open('rf_model.pkl', 'wb') as f:
    pickle.dump(rf, f)

print("RandomForest model saved as 'rf_model.pkl'")


In [24]:
 # load model
rf = pickle.load(open('rf.pkl','rb'))

In [ ]:
y_test = y_test.reset_index(drop=True)
print("Actual Disease :", y_test[0])


In [ ]:
# test 1:
print("predicted disease :",rf.predict(X_test.iloc[0].values.reshape(1,-1)))
print("Actual Disease :", y_test[0])

In [ ]:
# test 2:
print("predicted disease :",rf.predict(X_test.iloc[100].values.reshape(1,-1)))
print("Actual Disease :", y_test[100])

In [30]:
sym_des = pd.read_csv('/kaggle/input/symtoms_df.csv')
precautions = pd.read_csv('/kaggle/input/precautions_df.csv')
workout = pd.read_csv('/kaggle/input/workout_df.csv')
description = pd.read_csv('/kaggle/input/description.csv')
medications = pd.read_csv('/kaggle/input/medications.csv')
diets = pd.read_csv('/kaggle/input/diets.csv')


In [31]:
def helper(dis):
    desc = description[description['Disease'] == predicted_disease]['Description']
    desc = " ".join([w for w in desc])

    pre = precautions[precautions['Disease'] == dis][['Precaution_1', 'Precaution_2', 'Precaution_3', 'Precaution_4']]
    pre = [col for col in pre.values]

    med = medications[medications['Disease'] == dis]['Medication']
    med = [med for med in med.values]

    die = diets[diets['Disease'] == dis]['Diet']
    die = [die for die in die.values]

    wrkout = workout[workout['disease'] == dis] ['workout']


    return desc,pre,med,die,wrkout

symptoms_dict = {'itching': 0, 'skin_rash': 1, 'nodal_skin_eruptions': 2, 'continuous_sneezing': 3, 'shivering': 4, 'chills': 5, 'joint_pain': 6, 'stomach_pain': 7, 'acidity': 8, 'ulcers_on_tongue': 9, 'muscle_wasting': 10, 'vomiting': 11, 'burning_micturition': 12, 'spotting_ urination': 13, 'fatigue': 14, 'weight_gain': 15, 'anxiety': 16, 'cold_hands_and_feets': 17, 'mood_swings': 18, 'weight_loss': 19, 'restlessness': 20, 'lethargy': 21, 'patches_in_throat': 22, 'irregular_sugar_level': 23, 'cough': 24, 'high_fever': 25, 'sunken_eyes': 26, 'breathlessness': 27, 'sweating': 28, 'dehydration': 29, 'indigestion': 30, 'headache': 31, 'yellowish_skin': 32, 'dark_urine': 33, 'nausea': 34, 'loss_of_appetite': 35, 'pain_behind_the_eyes': 36, 'back_pain': 37, 'constipation': 38, 'abdominal_pain': 39, 'diarrhoea': 40, 'mild_fever': 41, 'yellow_urine': 42, 'yellowing_of_eyes': 43, 'acute_liver_failure': 44, 'fluid_overload': 45, 'swelling_of_stomach': 46, 'swelled_lymph_nodes': 47, 'malaise': 48, 'blurred_and_distorted_vision': 49, 'phlegm': 50, 'throat_irritation': 51, 'redness_of_eyes': 52, 'sinus_pressure': 53, 'runny_nose': 54, 'congestion': 55, 'chest_pain': 56, 'weakness_in_limbs': 57, 'fast_heart_rate': 58, 'pain_during_bowel_movements': 59, 'pain_in_anal_region': 60, 'bloody_stool': 61, 'irritation_in_anus': 62, 'neck_pain': 63, 'dizziness': 64, 'cramps': 65, 'bruising': 66, 'obesity': 67, 'swollen_legs': 68, 'swollen_blood_vessels': 69, 'puffy_face_and_eyes': 70, 'enlarged_thyroid': 71, 'brittle_nails': 72, 'swollen_extremeties': 73, 'excessive_hunger': 74, 'extra_marital_contacts': 75, 'drying_and_tingling_lips': 76, 'slurred_speech': 77, 'knee_pain': 78, 'hip_joint_pain': 79, 'muscle_weakness': 80, 'stiff_neck': 81, 'swelling_joints': 82, 'movement_stiffness': 83, 'spinning_movements': 84, 'loss_of_balance': 85, 'unsteadiness': 86, 'weakness_of_one_body_side': 87, 'loss_of_smell': 88, 'bladder_discomfort': 89, 'foul_smell_of urine': 90, 'continuous_feel_of_urine': 91, 'passage_of_gases': 92, 'internal_itching': 93, 'toxic_look_(typhos)': 94, 'depression': 95, 'irritability': 96, 'muscle_pain': 97, 'altered_sensorium': 98, 'red_spots_over_body': 99, 'belly_pain': 100, 'abnormal_menstruation': 101, 'dischromic _patches': 102, 'watering_from_eyes': 103, 'increased_appetite': 104, 'polyuria': 105, 'family_history': 106, 'mucoid_sputum': 107, 'rusty_sputum': 108, 'lack_of_concentration': 109, 'visual_disturbances': 110, 'receiving_blood_transfusion': 111, 'receiving_unsterile_injections': 112, 'coma': 113, 'stomach_bleeding': 114, 'distention_of_abdomen': 115, 'history_of_alcohol_consumption': 116, 'fluid_overload.1': 117, 'blood_in_sputum': 118, 'prominent_veins_on_calf': 119, 'palpitations': 120, 'painful_walking': 121, 'pus_filled_pimples': 122, 'blackheads': 123, 'scurring': 124, 'skin_peeling': 125, 'silver_like_dusting': 126, 'small_dents_in_nails': 127, 'inflammatory_nails': 128, 'blister': 129, 'red_sore_around_nose': 130, 'yellow_crust_ooze': 131}
diseases_list = {15: 'Fungal infection', 4: 'Allergy', 16: 'GERD', 9: 'Chronic cholestasis', 14: 'Drug Reaction', 33: 'Peptic ulcer diseae', 1: 'AIDS', 12: 'Diabetes ', 17: 'Gastroenteritis', 6: 'Bronchial Asthma', 23: 'Hypertension ', 30: 'Migraine', 7: 'Cervical spondylosis', 32: 'Paralysis (brain hemorrhage)', 28: 'Jaundice', 29: 'Malaria', 8: 'Chicken pox', 11: 'Dengue', 37: 'Typhoid', 40: 'hepatitis A', 19: 'Hepatitis B', 20: 'Hepatitis C', 21: 'Hepatitis D', 22: 'Hepatitis E', 3: 'Alcoholic hepatitis', 36: 'Tuberculosis', 10: 'Common Cold', 34: 'Pneumonia', 13: 'Dimorphic hemmorhoids(piles)', 18: 'Heart attack', 39: 'Varicose veins', 26: 'Hypothyroidism', 24: 'Hyperthyroidism', 25: 'Hypoglycemia', 31: 'Osteoarthristis', 5: 'Arthritis', 0: '(vertigo) Paroymsal  Positional Vertigo', 2: 'Acne', 38: 'Urinary tract infection', 35: 'Psoriasis', 27: 'Impetigo'}
def get_predicted_value(patient_symptoms):
    input_vector = np.zeros(len(symptoms_dict))
    for item in patient_symptoms:
        input_vector[symptoms_dict[item]] = 1
    return diseases_list[svc.predict([input_vector])[0]]

In [ ]:
import numpy as np
import pandas as pd

# Assuming symptoms_dict and diseases_list are defined like this:
# symptoms_dict = {'itching': 0, 'skin_rash': 1, 'dark_urine': 2, ...}
# diseases_list = ['Disease1', 'Disease2', 'Disease3', ...]

# Replace this with your actual trained classifier
# svc = RandomForestClassifier() or your trained model

# Function to get the predicted value
def get_predicted_value(patient_symptoms):
    input_vector = [0] * len(symptoms_dict)
    
    for item in patient_symptoms:
        if item in symptoms_dict:
            input_vector[symptoms_dict[item]] = 1
        else:
            print(f"Warning: Symptom '{item}' not recognized!")
    
    input_vector = np.array(input_vector).reshape(1, -1)
    
    predicted_index = rf.predict(input_vector)[0]
    
    if isinstance(predicted_index, int):
        return diseases_list[predicted_index]
    else:
        return predicted_index

# User input
symptoms = input("Enter your symptoms (comma-separated): ")
user_symptoms = [s.strip() for s in symptoms.split(',')]

# Predict the disease
predicted_disease = get_predicted_value(user_symptoms)

# Fetch additional details using the helper function
desc, pre, med, die, wrkout = helper(predicted_disease)

# Display Results
print("=================predicted disease============")
print(predicted_disease)
print("=================description==================")
print(desc)

print("=================precautions==================")
for i, p in enumerate(pre[0], start=1):
    print(f"{i}: {p}")

print("=================medications==================")
for i, m in enumerate(med, start=1):
    print(f"{i}: {m}")

print("=================workout==================")
for i, w in enumerate(wrkout, start=1):
    print(f"{i}: {w}")

print("=================diets==================")
for i, d in enumerate(die, start=1):
    print(f"{i}: {d}")


In [ ]:
print(selected_features.tolist())